Oliver Altindag

##*Pip*

In [ ]:
!pip install astroquery emcee corner

##*Data*

This section will load in the neccesary data from the M-Dwarf Catalog, as well as the needed Nasa Exoplanet data, with an added layer of entanglement by adding the matching Gaia DR2 and Gaia DR3 source ids. Additonally, it normalizes the M-Dwarf data and creates a corresponding dataframe.

In [ ]:
import numpy as np
import pandas as pd
import astropy
import astroquery
from astroquery.gaia import Gaia
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import emcee
import corner
from scipy import stats as st
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
import multiprocessing as mp
import seaborn as sns
import random
from matplotlib.patches import Patch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Df of the mdwarf data
data_mdwarf = pd.read_csv("/content/drive/Shareddrives/Project 3/Mdwarfcatalog.csv")
pd.set_option('display.max_columns', None)

In [ ]:
# Gets all of the known stellar hosts from nasa exoplanet archive
hosts_df = pd.read_csv("/content/drive/Shareddrives/Project 3/STELLARHOST.csv", skiprows = 8)
# Convert dr2_source_id to strings before applying string methods
hosts_df['gaia_id'] = hosts_df['gaia_id'].astype(str).str.replace("Gaia DR2 ", "", regex=False)
#Replaces the non-standard missing values with np.nan
hosts_df['gaia_id'] = hosts_df['gaia_id'].replace(["", "NaN", "None", "nan", "missing"], np.nan)
# Drop rows where 'gaia_id' is NaN
hosts_df = hosts_df.dropna(subset=['gaia_id'])
# makes it a string for later comparison
hosts_df['gaia_id'] = hosts_df['gaia_id'].astype(str)

In [ ]:
# Df of the mdwarf data
data_mdwarf = pd.read_csv("/content/drive/Shareddrives/Project 3/Mdwarfcatalog.csv")
pd.set_option('display.max_columns', None)

In [ ]:
# df of the crossmatched dr2 to dr3 ids
df_crossmatch = pd.read_csv("/content/drive/Shareddrives/Project 3/Crossmatch-result.csv")
pd.set_option('display.max_columns', None)
df_crossmatch['dr2_source_id'] = df_crossmatch['dr2_source_id'].astype(str) # assigns them all to string type so can calculate when they overlap bewteen df accurately

In [ ]:
# Select relevant columns, later renamed for aestetic purposes
abundance_columns = ['fe_h_cannon', 'mg_h_cannon', 'si_h_cannon',
                     'c_h_cannon', 'o_h_cannon']
error_columns = [col.replace('_h_cannon', '_h_err') for col in abundance_columns] # easier way for me to get the columns I needed

# Remove rows with missing values, keeping only the columns that are needed
# df_cleaned = data_mdwarf.dropna(subset=abundance_columns & error_columns) # original line
df_cleaned = data_mdwarf.dropna(subset=abundance_columns + error_columns) # using + to concatenate the lists instead of using the bitwise &

# Separate the source_id column
source_ids = df_cleaned['gaia_dr3_source_id'] # Now, extract the source_id column so it does not get normalized along with the rest of them

# Normalize the abundance columns, using sklearn
# This step was critical in using K-means in our clustering section of the document
# the only way to use a distance based algorithm like K-Means was to normalize them so each parameter was in the same scale
# if not the clusters would be overlapping, and our analyses would be invalid as it would be a comparison of identical data
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_cleaned[abundance_columns + error_columns]),
                             columns=abundance_columns + error_columns) # normalizing the data

# Add the source_id column BACK to the normalized DataFrame
df_normalized['source_id'] = source_ids.values

In [ ]:
# Finds the stellar hosts in our dataset
df_crossover = df_crossmatch[df_crossmatch['dr2_source_id'].isin(hosts_df['gaia_id'])]
# Defines the abundance columns
abundance_columns = ['fe_h_cannon', 'mg_h_cannon', 'si_h_cannon', 'c_h_cannon', 'o_h_cannon']

# Performs the merge
df_crossover = df_crossover.merge(
    df_normalized[['source_id'] + abundance_columns],  # Select only relevant columns (the abundance column we are analyzing)
    left_on='dr3_source_id',  # Match on Gaia DR3 source ID
    right_on='source_id',  # Match on Gaia ID in df_normalized (both from Gaia DR3)
    how='inner' # Keep only rows with matches in both DataFrames
)

# Drop the redundant source id, as there are two now
df_crossover.drop(columns=['source_id'], inplace=True, errors='ignore')

In [ ]:
# make the labels for each abundance more aestetically pleasing, will be needed in the plotting for the corner plots, and the 5x5 scatter plot of the normalized data bc we use f function
rename_dict = {
    'fe_h_cannon': '[Fe/H]', # All of these are just prettier reformations of the originals in proper abundance notation
    'mg_h_cannon': '[Mg/H]',
    'si_h_cannon': '[Si/H]',
    'c_h_cannon':  '[C/H]',
    'o_h_cannon':  '[O/H]',
}
df_normalized = df_normalized.rename(columns=rename_dict) # renames in the normalized data
df_crossover = df_crossover.rename(columns=rename_dict) # renames in the crossover data (the stellar hosts data)

##*Clustering*

We utilize a machine learning library, Sklearn, to create two statistically different populations based on the abundance data, along with an intermediary visual of the population means.

In [ ]:
# Creating two statistically DIFFERENT clusters based off of distance from the center of each cluster (centroid)
# 0.51 silhouette score difference which seemed to be significant enough (but need to report it)

# Fit K-Means with 2 clusters
kmeans = KMeans(n_clusters=2,random_state=11) # awesome random number choice with 11, same random state as in the mcmc

# Uses only the numeric columns (exclude source_id) for clustering
numeric_data = df_normalized.drop(columns=['source_id'])
numeric_data = numeric_data.drop(columns=error_columns) # bc dont want to factor the erorrs into the k-mean calculation (would also be trying to seperate the errors then)

# Adds labels from the column names
labels = kmeans.fit_predict(numeric_data)

# Add the cluster labels to df_normalized
df_normalized['cluster']=labels

# Compute the silhouette score (so we know how significant the difference is between them)
silhouette_avg = silhouette_score(numeric_data, labels)

print(f"Silhouette Score: {silhouette_avg:.2f}")

In [ ]:
# Compute mean properties for each cluster
# Used for visual inspection that the clusters are properly seperated, and as a reference point for usage in our mcmc as the prior
# Use df_normalized and refernce the new cluster column
cluster_summary = df_normalized.groupby('cluster')[['[Fe/H]', '[Mg/H]', '[Si/H]','[C/H]', '[O/H]']].mean() # chose these ones, which were outlines and renamed earlier as our selected columns
print("Cluster Summary (Mean):")
print(cluster_summary)

# Compute standard deviations
# Use df_normalized again referencing the cluster column
cluster_std = df_normalized.groupby('cluster')[['[Fe/H]', '[Mg/H]', '[Si/H]','[C/H]', '[O/H]']].std()
print("\nCluster Summary (Standard Deviation):")
print(cluster_std)

##*MCMC*

We use Bayesian inference and an MCMC platform, emcee, to extract the best fit parameters of the chemical abundance, accounting for uncertainty in measurement. Our logic is based on the principles of linear algebra, leveraging the properties of matrices to fit our multivariate model.

In [ ]:
abundance_columns = ['[Fe/H]', '[Mg/H]', '[Si/H]', '[C/H]', '[O/H]'] # same abundance columns as above, just referenceing them again for the code
error_columns = ['fe_h_err', 'mg_h_err', 'si_h_err', 'c_h_err', 'o_h_err'] # same abundance error columns as above, just referenceing them again for the code, and hard coded them

# Use the labels for clustering, bc need comparison per cluster
df_normalized['cluster'] =labels

# Split data by cluster for abundances and errors
clusters_abundance = {}
clusters_errors = {}

for cluster_id in df_normalized['cluster'].unique(): # splits them and adds them to the dictionary by cluster
    clusters_abundance[cluster_id]=df_normalized[df_normalized['cluster']==cluster_id][abundance_columns].values
    clusters_errors[cluster_id]=df_normalized[df_normalized['cluster']==cluster_id][error_columns].values

In [ ]:
# this takes 3 hours and change to run on my mobile workstation, we added the raw chains so it can be seen how it operates
# but I think on a mac this could potentially take 5 hours, so run it hesitantly, or just refernce the raw chains and corner plots to see results


# Short justification for our model and why we used each method:
# We parameterized the covariance matrix via its Cholesky decomposition to ensure positive definiteness while reducing free parameters and
# avoiding invalid proposals during sampling (pos def). Measurement errors are incorporated into the likelihood by augmenting the covariance
# matrix with diagonal error terms, preventing underestimating uncertainties or not including them at all. The log-likelihood used Cholesky-based
# Mahalanobis distances, the distance from the mean for each data point in standard deviation units, and log determinants for numerical robustness,
# using numpy linalg functions to avoid direct inverse calculations. We used weakly informative priors —Gaussian on means, log-uniform on variances,
# and Gaussian on off-diagonal Cholesky terms—regularizing the model while allowing data-driven inference. We used differnet functions for each bayesian parameter to
# help debug, and determine where errors were happening, and make easily adaptable chnages to the logic in our code.

# The Cholesky ensures valid covariance matrices and avoids numerical instability from direct inversion (pos def), which is critical for our high-dimensional
# analysis. The Mahalanobis distance captures correlations between variables (unlike Euclidean distance), ensuring the likelihood properly weights deviations
# from the mean relative to the full covariance structure, which bc our study is trying to find the best values given the uncertainty is necessary. Log determinants
# provide stable normalization without computing determinants directly, which would been computationally draining and more unstable. The multivariate Gaussian
# naturally unveiled itself and was adjusted to incorporate the necessary variables.

# A lot of the theory is in: https://files.eccomasproceedia.org/papers/uncecomp-2021/UC21_19074.pdf?mtime=20210921125442, but it is adapted from straight optimization to mcmc for the project
# Needed to acocunt for the fact that it recquires the full distribution not just gradient and other properties; the project is an extension of this general theory. This is why our code also
# contains normalization terms (which are not applicable in the optimization demonstrated in the paper bc the max posterior would be the same regardless). I am familiar with optimization like this, so the
# theory made logical sense when adapting; it is also much easier with mcmc, as they need an additional matrix and way more steps. Additionally, the cholesky decompositon was taught to me in class,
# if interested on how it operates more, it is on page 101 of this pdf: https://www.math.kent.edu/~reichel/courses/optimization/beck.pdf


# Set the random seed for reproducibility, my favorite number is 11 and does not make a difference what is in here
np.random.seed(11)

# Get unique cluster IDs from your normalized data, importatnt bc need to do this for each cluster individually
unique_clusters =df_normalized['cluster'].unique()

# Dictionary to store results for each cluster, will append after running to make all of our plots
results = {}

def process_cluster(cluster_id):
    """
    This function uses emcee to estimate the posterior distribution
    of the model parameters for a given cluster. The parameters are the mean vector (mu),
    the lower triangular matrix (L). Additionally, measurement errors are utilized in the liklihood function.
    The function returns the MCMC samples, the mean of the samples for the mean vector, the average covariance matrix,
    and raw chains.

    In:
        cluster_id (int): The ID of the cluster to process. This is used to filter the data
                          for the specific cluster from the dataset. There are internal fucntions inside, whch will call everything else needed.

    Out:
        tuple: A tuple containing the cluster ID and a dictionary with the following keys:
            - 'samples' (ndarray): The MCMC samples for the parameters.
            - 'mean_vector' (ndarray): The mean vector computed from the samples.
            - 'covariance_matrix' (ndarray): The covariance matrix computed from the samples.
            - 'raw_chains' (ndarray): The raw MCMC chains generated during the sampling process.
    """
    # Filter data for the specific cluster
    cluster_data = df_normalized[df_normalized['cluster']==cluster_id][abundance_columns].values # # Gets the needed data for given cluster, so that the calculations can be done
    cluster_errors = df_normalized[df_normalized['cluster']==cluster_id][error_columns].values # Gets the needed data for given cluster, so that the calculations can be done
    n_dimensions = cluster_data.shape[1] # finds the dimensions of the data by finding its shape, this will be used in almost every aspect of the model

    def log_likelihood(theta, data, errors):
      """
      Computes the log-likelihood for a multivariate Gaussian model with given abundance data and measurement errors.

      In:
          theta (array): Parameter vector containing mean and lower triangular matrix
          data (array): Data points for the abundance data
          errors (array): Measurement errors

      Out:
          float: Log-likelihood value or (-np.if to fail it).
      """
      mu =theta[:n_dimensions] # the mean vector
      L_flat =theta[n_dimensions:] # the lower triangular matrix, with the variables stored in a flat array
      L = np.zeros((n_dimensions, n_dimensions)) # Make a lower triangularmatrix with the proper shape
      L[np.tril_indices(n_dimensions)] =L_flat # add the values in to get a flled, properly shaped matrix
      Sigma_errorless= L @ L.T  # Shape: 5x5, the number of abuncances (was written before we knew which abundance columns so was made adaptive)

      try:
          # Batch compute total covariance for all stars, needed to use batch computation bc it does for all individually
          # Adds the data driven covariance mtx (Sigma_errorless) and the errors squared
          # this accounts for the measurement uncertainty and calculates a covaraince matrix accounting for uncertianty
          # tried to manually go about this but took eons to run so had to adjust to this using the einsum
          Sigma_total=Sigma_errorless+np.einsum('ij,jk->ijk',errors**2,np.eye(n_dimensions))

          # Cholesky (which is just what the lower triangular mtx is called) decomposition for stability, theres a lovely numpy function for this exact scenario
          # solves for the mtx by using the Sigma_total mtx whic now accunts for uncertainty
          # this alone saved minutes when doing testing with 1000 steps and less wlakers was an incredible find
          L_total=np.linalg.cholesky(Sigma_total)

          # Compute log-determinant from Cholesky
          # For a matrix A= L @ L.T the det(A) = (prod(diag(L)))^2 (theres a thm that det of lower triangular mtx is just product of the diagonals hence htis calculation)
          # so we can say log(det(A)) = 2*sum(log(diag(L)))
          logdet = 2*np.sum(np.log(np.diagonal(L_total,axis1=1,axis2=2)),axis=1)  # extracts the diagonals and does the above calculation

          # Compute residuals (this will be important in solving for the y variable)
          # broadcasting mu onto data
          diff =data-mu

          # Solve triangular systems for all stars
          # Numerically stable way to compute (Ly=diff), for y
          # Utilizes a numpy command so that do not need to manually do it, making this significantly faster
          # this is the same as doing y = L^-1 @ diff
          # then expand the dimesions so its correct dimensionally for the next calculation (will remove later, just temporary scaffolding)
          y =np.linalg.solve(L_total,diff[:, :,np.newaxis])

          # computes the chi square, which in this case is the mahalanobis distance, for EACH star
          # the suared norm of y gives the quadratic form (x-mu).T sigma⁻¹(x-mu), whoch can just be calculated by definition
          # removes the singleton dimension, bc we already used it in all the neccesary calc and simplifies future calculations
          chi2 =np.sum(y[:,:, 0]** 2,axis=1) #

          # Total log-likelihood
          # sum over all stars and return the negative
          # scaled to account for the volume of the data set (det.) and log(2npi) normalizes it for the high dimensionality
          return -0.5*np.sum(chi2 +logdet+n_dimensions*np.log(2 *np.pi)) # log multivariate gaussian, derived using the above variables in the context of the log of the multivariate pdf (non log version in the paper), summed all together bc for each individually
      except np.linalg.LinAlgError: # nope, not allowed so fail it, accuracy would be really bad if not
          return -np.inf # failing it


    def log_prior(theta):
        """
        Computes the log-prior for a multivariate Gaussian model

        In:
            theta (array): Parameter vector containing mean and lower triangular matrix

        Out:
            float: Log-prior value.
        """
        # essentially a repetition of what was done to initialize the log liklihood
        mu = theta[:n_dimensions] # mean vector from theta
        L_flat = theta[n_dimensions:] # flattened lower triangular mtx from theta
        L = np.zeros((n_dimensions, n_dimensions)) # Empty matrix to be filled
        L[np.tril_indices(n_dimensions)] =L_flat # filling said empty mtx
        logp_mu = -0.5 * np.sum(mu**2/10**2) # gaussina prior on the mean vector (log of the multi. gaussian pdf)
        logp_L = 0 # set the initialiaztion of log prior of L to be zero
        # looped the diagonal entries to ensure positivity
        for i in range(n_dimensions):
            if L[i, i]<=0: # removes non positive values
                return -np.inf # failing it with this
            # prior for the diagonals,
            logp_L +=-np.log(L[i, i]) # penalizes small values and ensures pos. definiteness
        # off diagonals
        logp_L +=-0.5*np.sum((L[np.tril_indices(n_dimensions, k=-1)])**2/5**2) # weak gaussina prior, does it only for the lower traingular entries
        return logp_mu+logp_L # the total prior is the diagonals and the non added together

    def log_posterior(theta):
        """
        Computes the log-posterior probability for the vector theta.

        The sum of the log-prior and the log-likelihood:
            log_posterior = log_prior + log_likelihood

        In:
            theta (array): Parameter vector containing mean and flattened lower triangular matrix

        Out:
            float: Log-posterior value. If the prior is invalid, returns -np.inf.
        """
        lp = log_prior(theta) # The prior for the given parameters
        if not np.isfinite(lp): # Fails impossible values, to ensure good results
            return -np.inf
        ll = log_likelihood(theta, cluster_data,cluster_errors) # the liklihood for the given parameters, as calculated by the function
        return lp + ll # Calculates the posterior, note bc log can add them insted of taking the product, better computationally

    # a repetition of what is done in the bayesian functions, but using the data for our initial steps
    initial_mu = np.mean(cluster_data,axis=0) # mean of each column
    initial_L= np.linalg.cholesky(np.cov(cluster_data, rowvar=False)) #  Takes the L amtrix, from the sample covariance matrix of the data, specific to the type of matrix i am using
    initial_L_flat =initial_L[np.tril_indices(n_dimensions)] # Flattens it (the above)
    initial_theta = np.concatenate([initial_mu, initial_L_flat]) # Combines them into a single array
    ndim = len(initial_theta) # Gets the dimensionality

    n_walkers = 50 # 2.5*ndim seemed adequete bc of the long run times, online i found a recommened 2-4 per dimension
    n_steps = 10000 # was told to use this for high quality data, although they converged wayyy sooner, very thorough which is good
    pos = initial_theta + 1e-4*np.random.randn(n_walkers,ndim) # the initial positon of the walkers and some noise incase it gets stuck

    # runs the mcmc
    sampler = emcee.EnsembleSampler(n_walkers, ndim, log_posterior)
    sampler.run_mcmc(pos, n_steps, progress=False)

    # Gets the chains to see if burn in was correct, and to impliment everything properly
    raw_chains = sampler.get_chain()
    samples = sampler.get_chain(discard=5000,thin=3, flat=True) # was terrified it would not converge after running a 1000 step test so really jacked this up bc how long it takes to run, doesnt really chnage anything in the end but was definity conesrvative

    # Collect samples for the covariance matrix
    Sigma_samples = []
    for sample in samples: # Iterate through flattened samples
        mu = sample[:n_dimensions] # the mean vector
        L_flat = sample[n_dimensions:] # just the values flattened
        L = np.zeros((n_dimensions, n_dimensions)) # just the mtx shape
        L[np.tril_indices(n_dimensions)] =L_flat # matrix wiht the values and with shape
        Sigma_samples.append(L @ L.T) #Calculates and store covariance matrix, as calculated from L above

    # these are all of the values added to the dictionary, and that will be used in visualizations (all except Figure 1)
    # the code below is juts how they are referenced
    return cluster_id, {
        'samples': samples,
        'mean_vector': np.mean(samples[:, :n_dimensions], axis=0),
        'covariance_matrix': np.mean(Sigma_samples, axis=0),
        'raw_chains': raw_chains
    }

# Parallel execution to process the data faster, set so it uses all cores
with mp.Pool(processes=mp.cpu_count()) as pool:
    result_list = pool.map(process_cluster,unique_clusters)

# Collect into dictionary which was made at the start of this cell, and can be called on later for analysis (used in graphing mcmc chains, corner plots, and heatmap)
# the values from the results portion
results = {cluster_id:result for cluster_id,result in result_list}

##*Visualization*

Utilizes the values above to plot the MCMC chains, along with 5x5 corner plots. Additionally, it includes the covariance and mean vector matrix derived from the MCMC. Finally, it includes a 5x5 matrix of the raw normalized abundances with overplot known stellar hosts, demonstrating how they fall in the clusters.

In [ ]:
# Print results for all clusters
for cluster_id, result in results.items(): # parses through the results dictionary
    print(f"\nResults for Cluster {cluster_id}:") # prints the cluster label so it is clear which is being analyzed
    print("Mean vector:")
    print(result['mean_vector']) # accesses the mean vector calculated in the mcmc
    print("Covariance matrix:")
    print(result['covariance_matrix']) # acceses the covariance matric calculated by the mcmc

In [ ]:
# Pretty labels (bc using the L matrix, not the abundance columns itself need to manually do this)
pretty_labels = ['[Fe/H]', '[Mg/H]', '[Si/H]', '[C/H]', '[O/H]']

# Create a figure with a 1x2 grid of subplots
fig, axes= plt.subplots(1, 2, figsize=(16,6))  # 1 row, 2 columns, adjust figsize as needed

# Sort the cluster keys so Cluster 0 is first, Cluster 1 is second
sorted_cluster_ids = sorted(results.keys())  # Ensures correct subplot order, as initially it plotted inverted bc the dictionary was updated with cluster 1 last

# Iterate through clusters and assign each heatmap to an axis
for i, cluster_id in enumerate(sorted_cluster_ids):  # Iterates through the clusters
    cov = results[cluster_id]['covariance_matrix']  # Gets the covariance matrix from the MCMC
    stddev =np.sqrt(np.diag(cov))  # Takes the STDs from the diagonal
    corr_matrix=cov/np.outer(stddev,stddev)  # Normalizes the covariance matrix to get correlations

    # Plot heatmap on the corresponding axis
    sns.heatmap(
        corr_matrix, # sets the data as the calculated correlation matrix
        annot=True, # displays the correlations in each box
        fmt=".2f", # The number of decimals
        cmap='coolwarm', # color map which matches with a later plot
        annot_kws={"fontsize": 14}, # so they are more readable
        cbar=True, # colorbar
        square=True, # shape
        linewidths=.5, # Aesthetics
        xticklabels=pretty_labels, # use the nice ones noted prior to the loop
        yticklabels=pretty_labels, # use the nice ones noted prior to the loop
        ax=axes[i]  # Assign the heatmap to the appropriate subplot
    )
    axes[i].set_title(f"Cluster {cluster_id}", fontsize=18) # Add a title to each subplot
    axes[i].tick_params(axis='both', labelsize=14) # to make it a bit more readable

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
# Retrieve the raw chains for Cluster 0
cluster_id = 0  # Specify its cluster 0 here
result = results[cluster_id]
raw_chains = result['raw_chains']

# Get the number of dimensions from the shape of the raw chains
n_dimensions = raw_chains.shape[2]  # Assuming the last dimension represents the parameters

# Separate the labels into mean vector and lower triangular matrix (L), as defned in the mcmc
mu_labels = abundance_columns # mean vector
L_labels = [f"L[{i},{j}]" for i in range(n_dimensions) for j in range(i + 1)] # L matrix

n_mu = len(mu_labels) # Number of parameters (this plot is only for the mean vectors)

# Plot the chains for the mean vector for Cluster 0
fig1, axes1 = plt.subplots(n_mu, figsize=(50, 15), sharex=True)

for i in range(n_mu): # Loops through the number of parameters, defined n_mu
    ax = axes1[i] # which axes to use, as deined by the i in the loop
    ax.plot(raw_chains[:, :, i], "k", alpha=0.3)  # Plot all walkers for parameter i
    ax.set_xlim(0, 10000) # The max step count, was easier to hard code in and ensured would look good
    ax.set_ylabel(mu_labels[i], fontsize=40) # Utilize the titles alreay in the dictionary, that is why we cleaned them upw ith "pretty_labels"
    ax.yaxis.set_label_coords(-0.05, 0.5)  # sets the postion of the y axis label, looked cramped and ugly before
    ax.tick_params(axis='x', which='major', length=20, width=5, labelsize=30)  # Major ticks
    ax.tick_params(axis='y', which='major', length=20, width=5, labelsize=30)  # Y-axis ticks

axes1[-1].set_xlabel("Steps",fontsize=50)
plt.tight_layout()
plt.show()

In [ ]:
# Retrieve the raw chains for Cluster 0
cluster_id = 1  # Specify its cluster 0 here
result = results[cluster_id]
raw_chains = result['raw_chains']

# Get the number of dimensions from the shape of the raw chains
n_dimensions = raw_chains.shape[2]  # Assuming the last dimension represents the parameters

# Separate the labels into mean vector and lower triangular matrix (L), as defned in the mcmc
mu_labels = abundance_columns # mean vector
L_labels = [f"L[{i},{j}]" for i in range(n_dimensions) for j in range(i + 1)] # L matrix

n_mu = len(mu_labels) # Number of parameters (this plot is only for the mean vectors)

# Plot the chains for the mean vector for Cluster 0
fig1, axes1 = plt.subplots(n_mu, figsize=(50, 15), sharex=True)

for i in range(n_mu): # Loops through the number of parameters, defined n_mu
    ax = axes1[i] # which axes to use, as deined by the i in the loop
    ax.plot(raw_chains[:, :, i], "k", alpha=0.3)  # Plot all walkers for parameter i
    ax.set_xlim(0, 10000) # The max step count, was easier to hard code in and ensured would look good
    ax.set_ylabel(mu_labels[i], fontsize=40) # Utilize the titles alreay in the dictionary, that is why we cleaned them upw ith "pretty_labels"
    ax.yaxis.set_label_coords(-0.05, 0.5)  # sets the postion of the y axis label, looked cramped and ugly before
    ax.tick_params(axis='x', which='major', length=20, width=5, labelsize=30)  # Major ticks
    ax.tick_params(axis='y', which='major', length=20, width=5, labelsize=30)  # Y-axis ticks

axes1[-1].set_xlabel("Steps",fontsize=50)
plt.tight_layout()
plt.show()

In [ ]:
cluster_id = 0  # cluster 0 is clarified here
result = results[cluster_id]
raw_chains = result['raw_chains']

# Get the number of dimensions from the shape of the raw chains
n_dimensions = raw_chains.shape[2]  # Assuming the last dimension represents the parameters

# Separate the labels into mean vector and lower triangular matrix (L)
mu_labels = abundance_columns
#changed to only use 5x5 here, like the covariance
L_labels = [f"L[{i},{j}]" for i in range(n_dimensions) for j in range(i + 1) if i < 5 and j < 5] #this one


# Number of parameters in each group, not need them both here os can refrnce everything properly
n_mu = len(mu_labels) # means vector length
n_L = len(L_labels) # covariance length

# Plot the chains for the lower triangular matrix (L) for Cluster 1
fig2, axes2 = plt.subplots(n_L, figsize=(50, 35), sharex=True)

for i in range(n_L): # bc this is for the covariances now
    ax = axes2[i] # again index to the i so it is put on the right axis, utilizing the loop function
    ax.plot(raw_chains[:, :, min(n_mu + i, raw_chains.shape[2] - 1)], "k", alpha=0.3)  # Plot all walkers for parameter i
    ax.set_xlim(0, 10000) # hard coded for efficiency and to make sure the plot ends at the correct place
    ax.set_ylabel(L_labels[i], fontsize=40) # uses just the referenced portion of the covariance matrix, hence the bracket notation, no nee dot make pretty bc for the apendix
    ax.yaxis.set_label_coords(-0.05, 0.5) # looked stupid as with just for the means so moved farther away
    ax.tick_params(axis='x', which='major', length=20, width=5, labelsize=30)  # Major ticks
    ax.tick_params(axis='y', which='major', length=20, width=5, labelsize=30)  # Y-axis ticks

axes2[-1].set_xlabel("Steps", fontsize=50) # sets the big x axis so it looked better thna having on each one
plt.tight_layout()
plt.show()

In [ ]:
cluster_id = 1  # cluster 1 this time around
result = results[cluster_id]
raw_chains = result['raw_chains']

# Get the number of dimensions from the shape of the raw chains
n_dimensions = raw_chains.shape[2]  # Assuming the last dimension represents the parameters

# Separate the labels into mean vector and lower triangular matrix (L)
mu_labels = abundance_columns
#changed to only use 5x5 here, like the covariance
L_labels = [f"L[{i},{j}]" for i in range(n_dimensions) for j in range(i + 1) if i < 5 and j < 5] #this one


# Number of parameters in each group, not need them both here os can refrnce everything properly
n_mu = len(mu_labels) # means vector length
n_L = len(L_labels) # covariance length

# Plot the chains for the lower triangular matrix (L) for Cluster 1
fig2, axes2 = plt.subplots(n_L, figsize=(50, 35), sharex=True)

for i in range(n_L): # bc this is for the covariances now
    ax = axes2[i] # again index to the i so it is put on the right axis, utilizing the loop function
    ax.plot(raw_chains[:, :, min(n_mu + i, raw_chains.shape[2] - 1)], "k", alpha=0.3)  # Plot all walkers for parameter i
    ax.set_xlim(0, 10000) # hard coded for efficiency and to make sure the plot ends at the correct place
    ax.set_ylabel(L_labels[i], fontsize=40) # uses just the referenced portion of the covariance matrix, hence the bracket notation, no nee dot make pretty bc for the apendix
    ax.yaxis.set_label_coords(-0.05, 0.5) # looked stupid as with just for the means so moved farther away
    ax.tick_params(axis='x', which='major', length=20, width=5, labelsize=30)  # Major ticks
    ax.tick_params(axis='y', which='major', length=20, width=5, labelsize=30)  # Y-axis ticks

axes2[-1].set_xlabel("Steps", fontsize=50) # sets the big x axis so it looked better thna having on each one
plt.tight_layout()
plt.show()

In [ ]:
# may need to split these os that we cna save them later, hypothetically an easy fix

# Plotting results for all clusters
for cluster_id, result in results.items(): # iterates through the clusters
    # Corner plot of the means
    samples = result['samples'] # gets the samples from the results dictionary
    mean_vector = result['mean_vector'] # gets the mean vector from the results dictionary
    print(f"\nResults for Cluster {cluster_id}:") # prints the cluster label so it is clear which is being analyzed
    corner.corner(
        samples[:, :len(mean_vector)], # Takes only the length of the mean vectors so it doesnt get the other parameters (only plotting the abundances nor covariance here)
        labels=abundance_columns, # uses the labels of the different abundances
        truths=mean_vector, # Adds refernce lines for the where the mean vectors are
        show_titles=True, # shows which is being compared, labels it for each abundance
        title_fmt=".3f", # three decimal places shown in the title (the writing on top of the histograms)
        title_kwargs={"fontsize": 12}, # wanted bigger so can read better in the report
        label_kwargs={"fontsize": 12}  # wanted bigger so can read better in the report, looked veyr small before this
    )
    plt.tight_layout()
    plt.show()

In [ ]:
abundance_columns = ['[Fe/H]', '[Mg/H]', '[Si/H]','[C/H]', '[O/H]'] #the only columns , but was not working when I referenced it from past cells so for convenience moved to here


fig, axes= plt.subplots(n, n, figsize=(20, 20)) # Sets the size of the plot, this seemed adequete and looked good

# Gets unique clusters and assign colors
clusters = np.sort(df_normalized['cluster'].unique())  # sorts to ensure Cluster 0 shows up first, cluster 1 was the last thing saved in the dictionary so important visual change in the labelling of this
colors = plt.cm.tab10(np.linspace(0, 1, len(clusters))) # keeps the coloring constatn throughout the plottign for each clustrer, letting me use one legend, and a fix from graphs in the last project, also a consistent color scheme here

for i in range(5): # Loop through the amount of different abundance colmumns
    for j in range(n): # Internal loop also going through all of the different abundaces so that I can reference the off diagonal and on diagonal plots, in a way manually making a coner plot
        ax =axes[i, j]  # Get the current subplot, critical to all of the fomulations to follow so they dont all overplot
        # Diagonals (want histograms of the twoclusters, bc the variable is being compared to itself, observing the overalp or lack thereof)
        if i == j: # how that is referenced in the matrix
            # Histograms for clusters
            for cluster,color in zip(clusters, colors): # loops through them, utilizing the assigned cluster and color (which are not in the loop)
                subset =df_normalized[df_normalized['cluster'] == cluster] # does it for the specicied clusters, then overplots them, called upon in the loop
                ax.hist(
                    subset[abundance_columns[i]], # does it for both clusters, referencing my last variable
                    bins=30, # 30 bins looked appropriate for the data
                    alpha=0.6, # stylistic choice
                    color=color # pre assigned, as noted
                )

            ax.set_xlabel(abundance_columns[i], fontsize=16) # sets an overaching label of what is being compared for each plot individually
            ax.set_ylabel("Frequency", fontsize=16) # will always be this, bc its a histogram

        # Non diagonals Scatter plots comparing two abundance columns
        else:
            # Scatter plots for clusters
            for cluster,color in zip(clusters, colors): # Loops through them, utilizing the assigned cluster and color again
                subset =df_normalized[df_normalized['cluster']== cluster] # gets the subset data needed, again this is going to be called upon in the plotting loop
                ax.scatter(
                    subset[abundance_columns[j]], # one axis is one abundance, using the variable subet to call it
                    subset[abundance_columns[i]], # one axis is another abundance, using subet again
                    alpha=0.5, # stylistic choice
                    s=10, # just right marker sizing
                    color=color # again using the consistent coloring
                )
            # Overplotting the stellar hosts (so can observe how they are distributed)
            ax.scatter(
                df_crossover[abundance_columns[j]], # one abundance columns on the x, no need for subets as all one dataset now, no loop like before
                df_crossover[abundance_columns[i]], # another abundance columnson the y
                alpha=0.8, # Looked nice with the red coloring
                s=15,
                color='red',
            )

            ax.set_xlabel(abundance_columns[j],fontsize=18) # Sets the fontize for the individual plots x axis labels, enlareged to make visible
            ax.set_ylabel(abundance_columns[i], fontsize=18) # Sets the fontsize for the y axis labels on the plots, to make readable

        # Adjust tick sizes to be readablish, also note they appear bizarrely scaled do the the normalization
        ax.tick_params(axis='both', which='major', labelsize=14)

# Add a legend for the entire figure, referencing the color assigned at the beginning
handles = [
    Patch(color=color,label=f"Cluster {cluster}") # refernces the color and links it to the proper cluster, uses patches in matplotlib, saved a significant amount of time, and is still comprehensible
    for cluster, color in zip(clusters, colors) # this is what makes it put BOTH clusters in the handle, without it will only do cluster 1 :(
]
handles.append(Patch(color='red',label="Stellar Hosts")) # adds another hnadle, which is not in the original color bank for the staller hosts, red is part of our color scheme

fig.legend(handles=handles,loc='upper center',bbox_to_anchor=(0.5, 1.02),ncol=len(clusters) + 1, fontsize=17) # sets the location to be directly down the middle

# Adjust layout
plt.tight_layout()
plt.show()

##*Statistical Comparison*

Utilizes a bootstrapped KS test to remedy the small number of known M-Dwarf stellar hosts in the Nasa Exoplanet archive, which determines if the rate of stellar hosts is from the same population bewteen the two clusters.

In [ ]:
# gets the source id from the two clusters, and makes them each a df of only those clusters
data1 = df_normalized[df_normalized['cluster']==0] # gets the data for cluster 0
data2 = df_normalized[df_normalized['cluster']==1] # gets the data for cluster 1

In [ ]:
# Convert all IDs to strings and strip whitespace, neccsary for finding the intersection
set1 = set(str(x).strip() for x in data1['source_id'])
set2= set(str(x).strip() for x in data2['source_id'])
set_hosts = set(str(x).strip() for x in df_crossover['dr2_source_id'])

# Finds the intersection between the two sets and the stellar hosts dataset
common_ids1 =set1.intersection(set_hosts)
common_ids2 = set2.intersection(set_hosts)

# Count the number of common IDs, again for the rate calculation
num_common_ids_all1 = len(common_ids1)
num_common_ids_all2 = len(common_ids2)

# The actual frequency calculation, dividing by the len of the set bc they are of diffrent lengths
freq_1 = num_common_ids_all1 / len(set1)
freq_2 = num_common_ids_all2 /len(set2)

print(f"Frequnecy of stellar hosts in cluster 0: {freq_1}")
print(f"Frequnecy of stellar hosts in cluster 1: {freq_2}")

In [ ]:
np.random.seed(11)
# Function to compute overlap counts for a given pair of sets
# make this a function, it gets really complicated if not in the loop referenced
def compute_overlap(set_a, set_b):
  """
  Computes the number of overlapping elements between two sets.

  In:
      set_a (set): First set cluster Gaia ids.
      set_b (set): Second set of stellar hostss Gaia ids.

  Out:
      int: The number of common elements between set_a and set_b.
  """
  return len(set_a.intersection(set_b)) # just gives the length of the intercetion, like the num_common_ids variables above

# Number of bootstrap iterations
n_bootstraps = 10000

# Initialize lists to store bootstrap overlap counts
bootstrap_counts1 = []
bootstrap_counts2 = []

# Perform bootstrapping
for i in range(n_bootstraps):
    # Randomly sample subsets of set1 and set2 (with replacement)
    sampled_set1 = set(np.random.choice(list(set1), size=len(set1), replace=True)) # randomly selects it, bootstrapping step to create the distribution
    sampled_set2 = set(np.random.choice(list(set2), size=len(set2), replace=True)) # randomly selects it, bootstrapping step to create the distribution

    # Compute the relative frequncy counts for the sampled sets, there will be 10,000 rate values in each list
    # look in the cell below to see this in action, and what the final distributions looks like
    bootstrap_counts1.append(compute_overlap(sampled_set1, set_hosts) / len(sampled_set1)) # overlap divided by the length of the set for each simulation round
    bootstrap_counts2.append(compute_overlap(sampled_set2, set_hosts) / len(sampled_set2)) # overlap divided by the length of the set for each simulation round

# Perform the KS test on the bootstrap distributions
ks_stat, p_value = ks_2samp(bootstrap_counts1, bootstrap_counts2)

# Print the results
print(f"KS Statistic: {ks_stat}")
print(f"P-Value: {p_value:.2e}")

# Interpretation, so we dont acidentaally mess this interpretation up later on, makes it explicit what this result means
if p_value < 0.05:
    print("The frequency of exoplanetary hosts is significantly different.")
else:
    print("The frequency of exoplanetary hosts is not significantly different.")

In [ ]:
# Cluster 0 rate dist.
print(bootstrap_counts1)

In [ ]:
# Cluster 1 rate dist
print(bootstrap_counts2)